In [2]:
import json

with open("Preprocessing_Wikipedia_Dataset.ipynb", "r", encoding="utf-8") as f:
    notebook = json.load(f)

# Safely remove widgets if exists
if "widgets" in notebook.get("metadata", {}):
    del notebook["metadata"]["widgets"]
    print("Removed metadata.widgets")

# Save cleaned version
with open("Preprocessing_Wikipedia_Dataset.ipynb", "w", encoding="utf-8") as f:
    json.dump(notebook, f, indent=2)


Removed metadata.widgets


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Redblock

/content/drive/MyDrive/Redblock


In [ ]:
%pip install sentencepiece tensorflow datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
import re
import random
import numpy as np
import tensorflow as tf
from datasets import load_dataset
import sentencepiece as spm
import os

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 12.6 MB/s eta 0:00:00


In [ ]:
# Load Wikipedia dataset
dataset = load_dataset("wikimedia/wikipedia", "20231101.en", split='train')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

train-00000-of-00041.parquet:   0%|          | 0.00/420M [00:00<?, ?B/s]

train-00001-of-00041.parquet:   0%|          | 0.00/351M [00:00<?, ?B/s]

...

train-00038-of-00041.parquet:   0%|          | 0.00/538M [00:00<?, ?B/s]

train-00039-of-00041.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

train-00040-of-00041.parquet:   0%|          | 0.00/422M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6407814 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm
import re
import unidecode

# List of unnecessary headings to be excluded
UNNECESSARY_HEADINGS = {
    "Discography", "Awards", "References", "External links",
    "External Links", "See also", "Further Reading", "Sources",
    "Bibliography", "Footnotes", "Filmography", "Gallery",
    "List of works", "Notable works,", "Cast"
}

headings = []
paragraphs = []
article_numbers = []

def normalize_to_ascii(text):
    """
    Normalizes text to ASCII using unidecode.
    Converts accented or special characters to their closest ASCII representation.
    """
    return unidecode.unidecode(text)

def preprocess_article(article, article_index):
    """
    Preprocesses a single article by breaking it down into headings and paragraphs.
    Normalizes text to ASCII during preprocessing.
    """
    # Normalize the entire article to ASCII
    article = normalize_to_ascii(article)

    # Remove everything beyond "References" or "External links"
    article = re.split(r'\n\nReferences|\n\nExternal links', article)[0]

    # Split article into sections based on double newlines
    sections = article.split('\n\n')

    current_heading = ""

    for section in sections:
        # Check if section has a heading followed by a newline and then content
        if '\n' in section:
            possible_heading, content = section.split('\n', 1)
            possible_heading = possible_heading.strip()

            # Normalize heading to ASCII
            possible_heading = normalize_to_ascii(possible_heading)

            # Adjusted threshold to 50 characters
            if len(possible_heading) <= 50:
                # Only process the heading if it is not in the unnecessary headings list
                if possible_heading not in UNNECESSARY_HEADINGS:
                    current_heading = possible_heading
                    paragraphs.append(normalize_to_ascii(content.strip()))  # Normalize content to ASCII
                    headings.append(current_heading)
                    article_numbers.append(article_index)
            else:
                # If it's not a heading, treat it as a paragraph without heading
                paragraphs.append(normalize_to_ascii(section.strip()))  # Normalize section to ASCII
                headings.append("")
                article_numbers.append(article_index)

        else:  # It's a standalone paragraph with no heading
            paragraphs.append(normalize_to_ascii(section.strip()))  # Normalize section to ASCII
            headings.append("")
            article_numbers.append(article_index)


# Process all articles
for index, article in tqdm(enumerate(articles), desc="Processing Articles", total=len(articles)):
    preprocess_article(article, index)


Processing Articles: 100%|██████████| 200000/200000 [02:43<00:00, 1219.91it/s]


In [ ]:
final_headings = []
content = []
article_no = []

current_content = ""
current_heading = None
current_article = None

for i in range(len(headings)):
    heading = headings[i]
    paragraph = paragraphs[i]
    article_index = article_numbers[i]

    if heading:  # If there's a heading
        if current_content:
            # Save the accumulated content and reset
            final_headings.append(current_heading)
            content.append(current_content.strip())
            article_no.append(current_article)

        # Start a new content block
        current_heading = heading
        current_content = paragraph
        current_article = article_index

    elif article_index == current_article:  # Continue accumulating if it's the same article
        current_content += "\n" + paragraph

    elif article_index != current_article:  # New article detected without a heading
        if current_content:  # Save the previous content block
            final_headings.append(current_heading if current_heading else "No Heading")
            content.append(current_content.strip())
            article_no.append(current_article)

        # Start a new block for the new article
        current_heading = "No Heading"
        current_content = paragraph
        current_article = article_index

# Save the last accumulated content
if current_content:
    final_headings.append(current_heading if current_heading else "No Heading")
    content.append(current_content.strip())
    article_no.append(current_article)


In [ ]:
import pickle

# File paths to save the arrays
final_headings_file = "final_headings.pkl"
headings_file = "headings.pkl"
content_file = "content.pkl"
paragraphs_file = "paragraphs.pkl"
article_no_file = "article_no.pkl"
article_numbers_file = "article_numbers.pkl"

# Save the 'final_headings' array
with open(final_headings_file, 'wb') as f:
    pickle.dump(final_headings, f)
print(f"Saved final_headings to {final_headings_file}")

# Save the 'headings' array
with open(headings_file, 'wb') as f:
    pickle.dump(headings, f)
print(f"Saved headings to {headings_file}")

# Save the 'content' array
with open(content_file, 'wb') as f:
    pickle.dump(content, f)
print(f"Saved content to {content_file}")

# Save the 'paragraphs' array
with open(paragraphs_file, 'wb') as f:
    pickle.dump(paragraphs, f)
print(f"Saved paragraphs to {paragraphs_file}")

# Save the 'article_no' array
with open(article_no_file, 'wb') as f:
    pickle.dump(article_no, f)
print(f"Saved article_no to {article_no_file}")

# Save the 'article_numbers' array
with open(article_numbers_file, 'wb') as f:
    pickle.dump(article_numbers, f)
print(f"Saved article_numbers to {article_numbers_file}")


Saved final_headings to final_headings.pkl
Saved headings to headings.pkl
Saved content to content.pkl
Saved paragraphs to paragraphs.pkl
Saved article_no to article_no.pkl
Saved article_numbers to article_numbers.pkl


In [ ]:
print(tokenizer.id_to_token(0))
print(tokenizer.id_to_token(1))
print(tokenizer.id_to_token(2))
print(tokenizer.id_to_token(3))

<pad>
<unk>
<s>
</s>


In [ ]:
for i in range(10):
    print("Final Heading: ", final_headings[i])
    print("Content: ", content[i])
    print("Article NUmber: ", article_no[i])

Final Heading:  No Heading
Content:  HMP Hull is a Category B men's local prison located in Kingston upon Hull in England. The term 'local' means that this prison holds people on remand to the local courts. The prison is operated by His Majesty's Prison Service.
Article NUmber:  0
Final Heading:  History
Content:  Hull Prison opened in 1870, and is of a typical Victorian design. 
Ethel Major was the last person and only woman to be executed at Hull in 1934. She had been convicted of the murder of her husband. An exhibition "Within These Walls" follows the prison's history from 1299 to 1934. 
The exhibition was designed and created by Officer Rob Nicholson and officially opened by Lawrence Major, Ethel's grandson.
In 1976 Hull prison was involved in a three-day riot by inmates of the prison. Over 100 prisoners were involved in a protest that erupted over staff brutality. The riot ended peacefully on 3 September 1976 but over two thirds of the prison was destroyed,
with an estimated repa

In [ ]:
articles[2]

'Victoria ("Tori") Freestone is a British saxophonist, flautist, violinist and composer. She has performed British jazz since 2009 as a band leader and sidewoman, known for her robust tenor sound and melodic invention. Her "Trio" albums, released in 2014 and 2016, were awarded at least 4 stars. The Guardian critic John Fordham described her first album "In The Chop House" as "an imposingly original sound".\nIn 2017 Freestone was shortlisted for a Fellowship in Jazz Composition supported by PRS for Music Foundation, UK Arts Foundation. That year Freestone was also nominated in the Parliamentary Jazz Awards 2017 in the Jazz Instrumentalist of the Year category.\n\nCareer\nFreestone started performing in folk clubs at the age of seven. She joined the National Youth Jazz Orchestra when she was 17 and then went on to study jazz flute at Leeds College of Music, then progressed to the saxophone when she was 26. Freestone likes to compose for unusual instrumentation that challenges traditional

## To Read Back the Files

In [ ]:
import pickle

# File paths to load the arrays
final_headings_file = "final_headings.pkl"
headings_file = "headings.pkl"
content_file = "content.pkl"
paragraphs_file = "paragraphs.pkl"
article_no_file = "article_no.pkl"
article_numbers_file = "article_numbers.pkl"

# Load the 'final_headings' array
with open(final_headings_file, 'rb') as f:
    final_headings = pickle.load(f)
print(f"Loaded final_headings from {final_headings_file}")

# Load the 'headings' array
with open(headings_file, 'rb') as f:
    headings = pickle.load(f)
print(f"Loaded headings from {headings_file}")

# Load the 'content' array
with open(content_file, 'rb') as f:
    content = pickle.load(f)
print(f"Loaded content from {content_file}")

# Load the 'paragraphs' array
with open(paragraphs_file, 'rb') as f:
    paragraphs = pickle.load(f)
print(f"Loaded paragraphs from {paragraphs_file}")

# Load the 'article_no' array
with open(article_no_file, 'rb') as f:
    article_no = pickle.load(f)
print(f"Loaded article_no from {article_no_file}")

# Load the 'article_numbers' array
with open(article_numbers_file, 'rb') as f:
    article_numbers = pickle.load(f)
print(f"Loaded article_numbers from {article_numbers_file}")


In [ ]:
import json

# Save articles to a JSON file
with open("wikipedia_articles_200k.json", "w", encoding="utf-8") as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

In [ ]:
len(paragraphs)

1730892

In [ ]:
len(content)

658212

# Train the Tokenizer

In [ ]:
import tensorflow as tf
import keras_hub
import os
from tqdm import tqdm

# Hyperparameters
VOCAB_SIZE = 20000  # As decided earlier
MODEL_TYPE = "bpe"  # We are using Byte Pair Encoding
PROTO_FILE = "tokenizer_bpe.proto"  # Output file for proto
SEQ_LEN = 128  # Sequence length for training

# Convert paragraphs to a TensorFlow Dataset
paragraphs_dataset = tf.data.Dataset.from_tensor_slices(paragraphs)

# Compute the SentencePiece proto using BPE model
proto = keras_hub.tokenizers.compute_sentence_piece_proto(
    paragraphs_dataset,
    vocabulary_size=VOCAB_SIZE,
    model_type=MODEL_TYPE,
    proto_output_file=PROTO_FILE,
    lowercase=False  # Since we have decided to keep the case as-is
)

In [ ]:
PROTO_FILE = "tokenizer_bpe.proto"

# Initialize tokenizer with SentencePiece model
tokenizer = keras_hub.tokenizers.SentencePieceTokenizer(
    proto=PROTO_FILE,
    sequence_length=SEQ_LEN,
    dtype="int32",
    add_bos=True,  # Automatically adds [BOS]
    add_eos=True   # Automatically adds [EOS]
)

In [ ]:
len(tokenizer.get_vocabulary())

20000

In [ ]:
i = 11
print("Original Paragraph:")
print(paragraphs[i])

# Tokenize the paragraph to get IDs
ids = tokenizer.tokenize(paragraphs[i])
print("\nToken IDs:")
print(ids)

# Convert IDs to tokens (ignoring padding tokens `0`)
decoded_tokens = [tokenizer.id_to_token(int(token)) for token in ids.numpy() if token != 0]
print("\nDecoded Tokens:")
print(decoded_tokens)

# Join the tokens to reconstruct the original sentence (for verification)
reconstructed_sentence = ' '.join(decoded_tokens)
print("\nReconstructed Sentence:")
print(reconstructed_sentence)


Original Paragraph:
HMP Hull houses the Within These Walls exhibition which charts the history of Hull's prisons from 1299 through to the present day. The exhibition was created by officer Rob Nicholson and opened in 2011.

Token IDs:
tf.Tensor(
[    2    67  6950 14430  4058    10  8568  1800   106  2333  5842   224
  5735    10  1481    26 14430 19973 19932   545 13048   143  1027  8205
   626    38    10  1296  1328 19946    69  5842    65  1790   117  3319
  2752  6067    57   524    33  1887    27  4792     3     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0

# Creating the Dataset

In [ ]:
import tensorflow as tf
import keras_hub

# Define constants
SEQ_LEN = 128

# Load the trained SentencePiece tokenizer
tokenizer = keras_hub.tokenizers.SentencePieceTokenizer(
    proto=PROTO_FILE,
    sequence_length=SEQ_LEN,
    dtype="int32",
    add_bos=True,
    add_eos=True
)

# Convert paragraphs to a TensorFlow Dataset
paragraphs_dataset = tf.data.Dataset.from_tensor_slices(paragraphs[:10])

# Define the preprocessing function
def preprocess(text):
    # Tokenize the text
    tokenized_text = tokenizer.tokenize(text)

    # Create inputs (features) and labels (targets)
    inputs = tokenized_text[:-1]  # Exclude the last token for inputs
    labels = tokenized_text[1:]   # Exclude the first token for labels

    # Return features and labels
    return inputs, labels

# Apply the preprocessing function to the dataset
train_ds = paragraphs_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

# Print a sample from the dataset to verify
for features, labels in train_ds.take(1):
    print("Features (Input Tokens):", features)
    print("Labels (Target Tokens):", labels)


Features (Input Tokens): tf.Tensor(
[    2    67  6950 14430    89     6 17169    58  1457 19973 19932  1135
  3049  1106    27 13940  2793 14430    27  1618 19946    69  1270   950
 19934  9663 19973  3364   136   377  3049  5635   799    77   616   114
    38    10  1135  7073 19946    69  3049    89  3536   117   980  7230
 10329 19973 19932 12393  3223 19946     3     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0], shape=(127,), dtype=int32)
Labels (Target Tokens): tf.Tensor(
[   67  6950 14430    89     6 17169    58  1457 19973 19932  1135  3049
  1106    27 13940  2793 14430    27  1618 19946    69

In [ ]:
# Create a new array that excludes entries with less than 80 characters
long_content = [entry for entry in content if len(entry) >= 80]

# Display count of the new array
print(f"Number of entries in 'long_content' (80 characters or more): {len(long_content)}")

# Optionally, you can print a few samples from the new array
print("Sample of entries in 'long_content':")
for i, entry in enumerate(long_content[:10]):  # Displaying first 10 entries
    print(f"{i+1}. {entry[:200]}...")  # Limiting the print length to 200 characters


Number of entries in 'long_content' (80 characters or more): 587898
Sample of entries in 'long_content':
1. HMP Hull is a Category B men's local prison located in Kingston upon Hull in England. The term 'local' means that this prison holds people on remand to the local courts. The prison is operated by His ...
2. Hull Prison opened in 1870, and is of a typical Victorian design. 
Ethel Major was the last person and only woman to be executed at Hull in 1934. She had been convicted of the murder of her husband. A...
3. Hull is a local prison holding remand, sentenced and convicted males. Prisoners are employed in the workshops, kitchens, gardens and waste management departments. Education classes are also available ...
4. Lionel Edgar Charles Letts BEM (15 August 1918 - 23 October 2013) was an English entrepreneur notable for a 75-year career in Southeast Asia during which he took a role on the boards of more than 90 l...
5. Letts was born in the English village of Send two months before 

In [ ]:
import os
import random

# Shuffle the long_content
random.shuffle(long_content)

# Split the data into training (90%) and validation (10%)
split_index = int(0.90 * len(long_content))
train_content = long_content[:split_index]
val_content = long_content[split_index:]

print(f"Number of training samples: {len(train_content)}")
print(f"Number of validation samples: {len(val_content)}")


Number of training samples: 529108
Number of validation samples: 58790


In [ ]:
output_dir = "data_files"  # Directory to save the files
os.makedirs(output_dir, exist_ok=True)

# File paths
train_file = os.path.join(output_dir, "train.txt")
val_file = os.path.join(output_dir, "valid.txt")

# Save the training content to a file
with open(train_file, "w", encoding="utf-8") as f:
    for entry in train_content:
        f.write(entry + "\n")

# Save the validation content to a file
with open(val_file, "w", encoding="utf-8") as f:
    for entry in val_content:
        f.write(entry + "\n")

print(f"Training data saved to: {train_file}")
print(f"Validation data saved to: {val_file}")


Training data saved to: data_files/train.txt
Validation data saved to: data_files/valid.txt


In [ ]:
len(train_content)

529108

In [ ]:
import tensorflow as tf

BATCH_SIZE = 64
SEQ_LEN = 128
MIN_STRING_LEN = 80  # This is our minimum length filter criterion

# Create the training dataset using tf.data.TextLineDataset.
raw_train_ds = (
    tf.data.TextLineDataset(train_file)
    .filter(lambda x: tf.strings.length(x) > MIN_STRING_LEN)
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

# Create the validation dataset.
raw_val_ds = (
    tf.data.TextLineDataset(val_file)
    .filter(lambda x: tf.strings.length(x) > MIN_STRING_LEN)
    .batch(BATCH_SIZE)
)




In [ ]:
import tensorflow as tf

num_samples = tf.data.experimental.cardinality(raw_train_ds).numpy()
print(f"Number of samples in train_ds: {num_samples}")

Number of samples in train_ds: -2


In [ ]:
# Displaying the number of lines loaded from each dataset
num_train_samples = sum(1 for _ in raw_train_ds)
num_val_samples = sum(1 for _ in raw_val_ds)
print(f"Number of training samples loaded: {num_train_samples}")
print(f"Number of validation samples loaded: {num_val_samples}")

In [ ]:
import tensorflow as tf
import keras_hub

# Define constants
SEQ_LEN = 129

# Load the trained SentencePiece tokenizer
tokenizer = keras_hub.tokenizers.SentencePieceTokenizer(
    proto=PROTO_FILE,
    sequence_length=SEQ_LEN,
    dtype="int32",
    add_bos=True,
    add_eos=True
)

def preprocess(inputs):
    # Tokenize the input text using the SentencePiece tokenizer
    outputs = tokenizer.tokenize(inputs)

    # Create features and labels - shifting the tokens
    features = outputs[:, :-1]  # All tokens except the last one
    labels = outputs[:, 1:]     # All tokens except the first one

    return features, labels

# Tokenize and split into train and label sequences.
train_ds = raw_train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
val_ds = raw_val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

In [ ]:
for features, labels in train_ds.take(1):
    print("Features (Input Tokens):", features)
    print("Labels (Target Tokens):", labels)

Features (Input Tokens): tf.Tensor(
[[    2    79  1560 ...     0     0     0]
 [    2  1539    26 ...     0     0     0]
 [    2   134 19958 ...     0     0     0]
 ...
 [    2    44  7441 ...     0     0     0]
 [    2   367 17355 ... 19943   379   589]
 [    2  1448    10 ...    26  2347   105]], shape=(64, 128), dtype=int32)
Labels (Target Tokens): tf.Tensor(
[[   79  1560  4236 ...     0     0     0]
 [ 1539    26    10 ...     0     0     0]
 [  134 19958   170 ...     0     0     0]
 ...
 [   44  7441    26 ...     0     0     0]
 [  367 17355    26 ...   379   589  4716]
 [ 1448    10   261 ...  2347   105   117]], shape=(64, 128), dtype=int32)


In [ ]:
import tensorflow as tf
import os

# Define directory paths for saving datasets
output_dir = "saved_datasets"
train_dir = os.path.join(output_dir, "train_ds")
val_dir = os.path.join(output_dir, "val_ds")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save train_ds and val_ds
tf.data.experimental.save(train_ds, train_dir)
tf.data.experimental.save(val_ds, val_dir)

print(f"Training dataset saved at: {train_dir}")
print(f"Validation dataset saved at: {val_dir}")


Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


Training dataset saved at: saved_datasets/train_ds
Validation dataset saved at: saved_datasets/val_ds


In [ ]:
# Reloading train_ds and val_ds
loaded_train_ds = tf.data.experimental.load(train_dir)
loaded_val_ds = tf.data.experimental.load(val_dir)

print("Training dataset loaded successfully!")
print("Validation dataset loaded successfully!")


Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


Training dataset loaded successfully!
Validation dataset loaded successfully!


In [ ]:
for features, labels in loaded_train_ds.take(1):
    print("Features (Input Tokens):", features)
    print("Labels (Target Tokens):", labels)

Features (Input Tokens): tf.Tensor(
[[    2  1997   134 ...    69   198   392]
 [    2    69 12554 ...  5131  2883 19946]
 [    2    69  6170 ...     0     0     0]
 ...
 [    2  5825    33 ...     0     0     0]
 [    2  3362    78 ...     0     0     0]
 [    2 15311   123 ...     0     0     0]], shape=(64, 128), dtype=int32)
Labels (Target Tokens): tf.Tensor(
[[ 1997   134 19953 ...   198   392  1120]
 [   69 12554 16503 ...  2883 19946   135]
 [   69  6170 10860 ...     0     0     0]
 ...
 [ 5825    33   107 ...     0     0     0]
 [ 3362    78  3474 ...     0     0     0]
 [15311   123   386 ...     0     0     0]], shape=(64, 128), dtype=int32)
